<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Простые-модели" data-toc-modified-id="Простые-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Простые модели</a></span></li><li><span><a href="#Модели-градиентного-бустинга" data-toc-modified-id="Модели-градиентного-бустинга-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модели градиентного бустинга</a></span></li><li><span><a href="#Проверим-модель-на-тестовой-выборке" data-toc-modified-id="Проверим-модель-на-тестовой-выборке-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Проверим модель на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

In [27]:
import imblearn
import numpy as np
import pandas as pd
import re
import transformers as ppb
import torch

from collections import Counter
from catboost import CatBoostClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.utils.class_weight import compute_class_weight
from tqdm import notebook
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')


In [28]:
# инициализируем токенизатор
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# экземпляр модели
model = model_class.from_pretrained(pretrained_weights)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
# Функция для очистки текста от лишних символов
def clean_text(text):
    # приводим текст к нижнему регистру
    text = text.lower()
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    # регулярное выражение для замены ссылки на "URL"
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # заменяем ссылки на "URL"
    text = re.sub(regular_url, r'URL', text)
    # заменяем числа и цифры на ' NUM '
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # возвращаем очищенные данные
    return text
 
# создаем список для хранения очищенных данных
cleaned_text = []
# для каждого сообщения text из столбца data['text']
for text in df['text']:
    # очищаем данные  
    text = clean_text(text)
    # добавляем очищенные данные в список cleaned_text
    cleaned_text.append(text)
# записываем очищенные данные в новую колонку
df['text'] = cleaned_text

df.info()
display(df['toxic'].value_counts())
display(df)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 3.0+ MB


0    143106
1     16186
Name: toxic, dtype: int64

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww! he matches this background colour i'm s...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on imp...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159446,and for the second time of asking when your vi...,0
159447,you should be ashamed of yourself that is a ho...,0
159448,spitzer umm theres no actual article for prost...,0
159449,and it looks like it was actually you who put ...,0


In [30]:
df = df.sample(400).reset_index(drop=True)

batch_1 = df.copy()
batch_1['tokenized_text'] = batch_1.text.apply(lambda x: tokenizer.encode(x,
                                                                          add_special_tokens=True,
                                                                          truncation=True,
                                                                          max_length=512
                                                                         )
                                               )

batch_1['tokenized_len'] = batch_1['tokenized_text'].apply(len)
display(batch_1.head())
print(batch_1['toxic'].value_counts())


,text,toxic,tokenized_text,tokenized_len
0,unprotect my page i am off my block and theref...,0,"[101, 4895, 21572, 26557, 2102, 2026, 3931, 10...",56
1,siberian kyiv post goble sept NUM duplicate ge...,0,"[101, 21822, 25669, 2695, 2175, 3468, 17419, 1...",56
2,i rewrote the intro today changing passivelyvo...,0,"[101, 1045, 2128, 13088, 12184, 1996, 17174, 2...",171
3,uscentric the layout of this article seems to...,0,"[101, 15529, 4765, 7277, 1996, 9621, 1997, 202...",90
4,please be civil! how many times have you been ...,0,"[101, 3531, 2022, 2942, 999, 2129, 2116, 2335,...",70


0    357
1     43
Name: toxic, dtype: int64


In [31]:
# токенизированный текст
tokenized = batch_1['tokenized_text']

# применим padding к векторам
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# англ. вектор с отступами
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)
print('attention_mask:',attention_mask.shape)

batch_size = 100

# создаем эмбеддинги
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())


attention_mask: (400, 512)


  0%|          | 0/4 [00:00<?, ?it/s]

## Обучение

### Простые модели

In [32]:
# Разделим на три выборки
features = np.concatenate(embeddings)
features_train, features_valid = train_test_split(features, shuffle=False, test_size=0.2)
features_valid, features_test = train_test_split(features_valid, shuffle=False, test_size=0.4)

target_train, target_valid = train_test_split(df['toxic'], shuffle=False, test_size=0.2)
target_valid, target_test = train_test_split(target_valid, shuffle=False, test_size=0.4)

In [33]:
%%time
y_pred = LogisticRegression(class_weight='balanced').fit(features_train, target_train).predict(features_valid)
f1_score(target_valid, y_pred)

Wall time: 22.9 ms


0.4444444444444445

In [34]:
%%time

best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(class_weight='balanced', random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = f1_score(target_valid, y_pred)
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("RMSE DecisionTreeRegressor:", best_result,
      "\nГлубина дерева:", best_depth)


RMSE DecisionTreeRegressor: 0.4444444444444445 
Глубина дерева: 1
Wall time: 187 ms


In [35]:
%%time
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(class_weight='balanced', random_state=12345, n_estimators=est,
                                      max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = f1_score(target_valid, y_pred)
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("RMSE RandomForestRegressor:", best_result,
      "Количество деревьев:", best_est,
      "Максимальная глубина:", best_depth)


RMSE RandomForestRegressor: 0.4444444444444445 Количество деревьев: 10 Максимальная глубина: 1
Wall time: 2.5 s


### Модели градиентного бустинга

In [20]:
# Разделим на две выборки
features_train, features_test = train_test_split(features, shuffle=False, test_size=0.1)
target_train, target_test = train_test_split(df['toxic'], shuffle=False, test_size=0.1)

# веса классов для моделей градиентного бустинга
classes = np.unique(target_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=target_train)
class_weights = dict(zip(classes, weights))

In [15]:
# фунция всех операций и результата моделей градиентного бустинга
def result(estimator, rf_grid):
        
    model = GridSearchCV(estimator, rf_grid,
                         scoring='f1',
                         n_jobs=5, cv=3)
    
    result = model.fit(features_train, target_train)

    print(f'{str(estimator)}. Best Hyperparameters: %s' % result.best_params_)
    y_pred = model.predict(features_test)
    predictions = [round(value) for value in y_pred]
    
    print(f'{str(estimator)}. F1: %.2f' %
          f1_score(target_test, y_pred))

In [16]:
%%time
param = {'n_estimators': [200, 300, 500, 700, 1000],
           "min_child_weight": [1, 3, 5, 10, 25],
           'max_depth': [3, 4, 7, 10, 25, 50]
           }
result(LGBMClassifier(random_state=12345, class_weight = class_weights), param)


LGBMClassifier(class_weight={0: 0.5487804878048781, 1: 5.625}). Best Hyperparameters: {'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 200}
LGBMClassifier(class_weight={0: 0.5487804878048781, 1: 5.625}). F1: 0.57
Wall time: 31.6 s


In [17]:
%%time
param = {'max_depth': [10, 25, 50, 75, 100, 200],
         'min_child_samples': [1, 2, 3]
         }
result(CatBoostClassifier(class_weights=class_weights, random_state=12345, iterations=10), param)

Learning rate set to 0.454439
0:	learn: 0.3897847	total: 934ms	remaining: 8.41s
1:	learn: 0.2566630	total: 1.71s	remaining: 6.84s
2:	learn: 0.1851904	total: 2.49s	remaining: 5.8s
3:	learn: 0.1350879	total: 3.26s	remaining: 4.88s
4:	learn: 0.1031826	total: 4.03s	remaining: 4.03s
5:	learn: 0.0826828	total: 4.79s	remaining: 3.19s
6:	learn: 0.0691652	total: 5.58s	remaining: 2.39s
7:	learn: 0.0592042	total: 6.36s	remaining: 1.59s
8:	learn: 0.0521740	total: 7.14s	remaining: 794ms
9:	learn: 0.0445356	total: 7.91s	remaining: 0us
<catboost.core.CatBoostClassifier object at 0x000001A64C51AA90>. Best Hyperparameters: {'max_depth': 10, 'min_child_samples': 1}
<catboost.core.CatBoostClassifier object at 0x000001A64C51AA90>. F1: 0.50
Wall time: 1min 11s


In [21]:
%%time
param_grid = {"max_depth": [3, 4, 7, 10, 25],
              "gamma": [0.5, 1, 5, 10, 25],
              "min_child_weight": [1, 3, 5, 10, 25]
             }
result(XGBClassifier(class_weight=class_weights, verbosity = 0), param)


XGBClassifier(class_weight={0: 0.5487804878048781, 1: 5.625}, verbosity=0). Best Hyperparameters: {'max_depth': 10, 'min_child_samples': 1}
XGBClassifier(class_weight={0: 0.5487804878048781, 1: 5.625}, verbosity=0). F1: 0.33
Wall time: 14 s


### Проверим модель на тестовой выборке

In [38]:
%%time
model = LGBMClassifier(max_depth=10, min_child_weight=1, n_estimators=200)
model.fit(features_train, target_train)

predictions_train = model.predict(features_train)
predictions_test = model.predict(features_test)

clf = DummyClassifier(strategy='most_frequent', random_state=12345)
clf.fit(features_train, target_train)
print("Наилучшая модель - LightGBM")
print("F1 на тестовой выборке: %.2f" %
      f1_score(target_test, predictions_test)**.5)

Наилучшая модель - LightGBM
F1 на тестовой выборке: 0.82
Wall time: 486 ms


## Выводы

- Загрузили и подготовили набор данных с разметкой о токсичности правок. Очистили текст от лишних символов. Инициализировали токенизатор BERT.
- Разделили набор данных на выборки. Обучили на них как простые модели так и с градиентным бустингом.
- По итогам тестов, определили лучшую модель по метрике F1 на трех выборках: LightGBM. Обучили модель классифицировать комментарии на позитивные и негативные с качеством F1 более 0.75.